# Создание новых признаков

In [1]:
# подгрузим необходимые нам библиотеки
import pandas as pd 
import numpy as np

Перед выполнением машинного обучения крайне важный оказывается рассмотрение уже имеющихся признаков, а также работа с ними. Нередко добавление новых осознанных признаков на основе уже имеющихся характеристик способно улучшить качество модели за счет предоставления большей инофрмации об оцениваемых объектах. В этой части проекта мы сфокусируемся непосредственно на придумывании новых признаков, выделенных с помощью ранее выявленных закономерностях в данных.

In [2]:
cars_df = pd.read_csv('cars_df_interim.csv')
cars_df.head()

,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,1-owner vehicle,Personal use only,MPG_min,MPG_max,MPG_mean,Mileage,Price
0,Hyundai,2022,Red,Black,Front-wheel Drive,Gasoline,2.5,I4,16.0,At least 1 accident or damage reported,Yes,Yes,22.000000,28.000000,25.000000,13256.0,38988.0
1,Lexus,2016,Black,Black,All-wheel Drive,Gasoline,3.5,V6,24.0,At least 1 accident or damage reported,No,Yes,19.000000,26.000000,22.500000,100067.0,19747.0
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,At least 1 accident or damage reported,Yes,No,20.592777,27.239548,23.916163,39032.0,25999.0
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,None reported,Yes,No,20.592777,27.239548,23.916163,10901.0,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,None reported,Yes,Yes,17.000000,22.000000,19.500000,24946.0,61530.0


__1)__ *Usage Intensity* - интенсивность использования автомобиль с момента его выпуска. Рассчитывается как отношение общего пробега к количеству лет использования.

In [3]:
# Будем считать год, если автомобиль в течение него использовался
cars_df['Usage Intensity'] = cars_df['Mileage'] / (2023 - cars_df['Year'] + 1)

---

__2)__  *Ownership History* - комбинированный признак, учитывающий информацию о количестве владельцев автомобиля (1-owner vehicle) и наличии аварий или повреждений (Accidents or damage). Принимает три значения значения: "Good" - один владелец без аварий и повреждений, "Potential risk" - несколько владельцев или наличие аварий/повреждений, а также "Bad" - несколько владельцев или наличие аварий/повреждений.

In [4]:
conditions = [
    (cars_df['1-owner vehicle'] == 'Yes') & (cars_df['Accidents or damage'] == 'None reported'),
    (cars_df['1-owner vehicle'] == 'No') & (cars_df['Accidents or damage'] == 'At least 1 accident or damage reported')
]

values = ['Good', 'Bad']

cars_df['Ownership History'] = np.select(conditions, values, default='Potential Risk')

__3)__ *Engine Power* - информация о мощности двигателя с учетом  его размера и количества клапанов. Расчитывается с помощью  умножения емкости двигателя (Capacity) на количество клапанов (Valves).

In [5]:
cars_df['Engine Power'] = cars_df['Capacity'] * cars_df['Valves']

__4)__ *Fuel Efficiency* - эффективность использования топлива в зависимости от размера двигателя. Вычисляется путем деления среднего значения удельного израсхода топлива (MPG_mean) на емкость двигателя (Capacity). Новые признак играет важную роль в описании оптимальности использования ресурсов. 

In [6]:
cars_df['Fuel Efficiency'] = cars_df['MPG_mean'] / cars_df['Capacity']

__5)__ *Origin* - признак, обозначающий происхождение автомобиля.

In [7]:
# Получим списки автомобилей по происхождению
american_cars = ['Chevrolet', 'Chrysler', 'GMC', 'Cadillac',
                 'Ford', 'Buick', 'Jeep', 'Lincoln', 'RAM',
                 'Pontiac', 'Dodge', 'Hummer', 'Plymouth']

european_cars = ['Mercedes-Benz', 'Audi', 'BMW', 'Lamborghini', 'Porsche', 'Volkswagen',
                 'Volvo', 'FIAT', 'MINI', 'Maserati', 'Alfa Romeo', 'Ferrari', 'Saab']

british_cars = ['Land Rover', 'McLaren', 'Jaguar', 'Aston Martin', 'Bentley', 'Lotus', 'Rolls-Royce']

asian_cars = ['Hyundai', 'Lexus', 'INFINITI', 'Mazda', 'Toyota', 'Genesis', 'Kia',
              'Acura', 'Nissan', 'Honda', 'Mitsubishi', 'Scion', 'Suzuki']

origin_dict = {}
for brand in american_cars:
    origin_dict[brand] = 'American'
    
for brand in european_cars:
    origin_dict[brand] = 'European'

for brand in british_cars:
    origin_dict[brand] = 'British'

for brand in asian_cars:
    origin_dict[brand] = 'Asian'

In [8]:
cars_df['Origin'] = np.vectorize(lambda x: origin_dict[x])(cars_df['Brand'])

__6)__ *Gallons Per Year* - признак, показывающий, сколько топлива в среднем в год требует автомобиль. Так как MPG - Miles Per Gallon - количество миль, которое автомобиль сможет проехать на одном галлоне топлива, то общее число потраченных галлонов равно отношению Mileage к MPG. Тогда количество галлонов в год находится делением общего числа использованных галлонов на количество лет эксплуатации.

In [9]:
cars_df['Gallons Per Year'] = (cars_df['Mileage'] / (cars_df['MPG_mean']) / (2023 - cars_df['Year'] + 1))

---

In [10]:
# Перенесем столбец с ценами в конец таблицы
price = cars_df.pop('Price')
cars_df['Price'] = price
cars_df.head()

,Brand,Year,Exterior color,Interior color,Drivetrain,Fuel type,Capacity,Configuration,Valves,Accidents or damage,...,MPG_max,MPG_mean,Mileage,Usage Intensity,Ownership History,Engine Power,Fuel Efficiency,Origin,Gallons Per Year,Price
0,Hyundai,2022,Red,Black,Front-wheel Drive,Gasoline,2.5,I4,16.0,At least 1 accident or damage reported,...,28.000000,25.000000,13256.0,6628.000,Potential Risk,40.0,10.000000,Asian,265.120000,38988.0
1,Lexus,2016,Black,Black,All-wheel Drive,Gasoline,3.5,V6,24.0,At least 1 accident or damage reported,...,26.000000,22.500000,100067.0,12508.375,Bad,84.0,6.428571,Asian,555.927778,19747.0
2,Chevrolet,2017,White,Black,Front-wheel Drive,Hybrid,1.5,I4,16.0,At least 1 accident or damage reported,...,27.239548,23.916163,39032.0,5576.000,Potential Risk,24.0,15.944109,American,233.147769,25999.0
3,Mercedes-Benz,2022,White,Other,All-wheel Drive,Gasoline,3.0,I6,24.0,None reported,...,27.239548,23.916163,10901.0,5450.500,Good,72.0,7.972054,European,227.900272,96750.0
4,Land Rover,2020,Blue,Other,Four-wheel Drive,Gasoline,3.0,I6,24.0,None reported,...,22.000000,19.500000,24946.0,6236.500,Good,72.0,6.500000,British,319.820513,61530.0


In [11]:
# Сохраним итоговую таблицу
cars_df.to_csv('cars_df_final.csv', index=False)